# 金融 证券 图谱搭建

## 1. 图数据库设置

### 1.1创建索引

提高查询效率

In [26]:
index_scripts = """CREATE INDEX ON :CITY(city);
                CREATE INDEX ON :FUND(fund_code);
                CREATE INDEX ON :COMPANY(company);
                CREATE INDEX ON :FUND_CUSTODIAN(company);
                CREATE INDEX ON :FUND_MANAGER(company);
                CREATE INDEX ON :LISTED_COMPANY(company);
                CREATE INDEX ON :INDUSTRY(industry);
                CREATE INDEX ON :MANAGER(manager_id);
                CREATE INDEX ON :PROVINCE(province);
                CREATE INDEX ON :LISTED_COMPANY(share_code);
                """

In [27]:
for scripts in index_scripts.split('\n'):
    scripts = scripts.strip()
    if len(scripts)>5 :
        print(scripts)
        result = graph.run(scripts)

CREATE INDEX ON :CITY(city);
CREATE INDEX ON :FUND(fund_code);
CREATE INDEX ON :COMPANY(company);
CREATE INDEX ON :FUND_CUSTODIAN(company);
CREATE INDEX ON :FUND_MANAGER(company);
CREATE INDEX ON :LISTED_COMPANY(company);
CREATE INDEX ON :INDUSTRY(industry);
CREATE INDEX ON :MANAGER(manager_id);
CREATE INDEX ON :PROVINCE(province);
CREATE INDEX ON :LISTED_COMPANY(share_code);


### 1.2. 安装插件

主要是 apoc 插件。这里使用的是 `apoc-3.4.0.3-all.jar `

## 2. 知识图谱应用

下面是应用部分。主要从两个方面展开
1. 可视化

利用 neo4j 自带的可视化插件进行

2. 关联查询

利用 neo4j 的查询语言 `Cypher` 进行相关查询。

In [12]:
import os
from py2neo import Graph
import warnings
warnings.filterwarnings('ignore')

In [13]:
def check_result(result):
    for c in result :
        print(c)

In [14]:
graph = Graph(
    "bolt://localhost:7687",
    usernmae="neo4j",
    password="123456"
)

### 2.1.可视化

关系图谱将数据以关联方式呈现，更直观。正所谓一图胜千言

这里举个例子：
**查看山东国资背景的上市公司，在城市和行业上的分布情况。**

国资背景上市公司
https://zhuanlan.zhihu.com/p/22894178

山东共有省属企业23家，资产总额12906亿元。其中部分上市公司的有：
- 山东钢铁（600022）
- 山东黄金（600547）
- 鲁商置业（600223）
- 银座股份（600858）
- 山东高速（600350）
- 山东路桥（000498）
- 潍柴动力（000338）
- 浪潮集团（000977）
- 浪潮软件（600756）
- 鲁银投资（600784）
- 新华制药（000756）
- 鲁抗医药（600789）

In [15]:
query = """
        MATCH p1 = (a:LISTED_COMPANY)-[:IN_INDUSTRY]-(b:INDUSTRY) 
            WHERE a.share_code in ["600022","600547","600223","600858","600350","000498","000338","000977","600756","600784","000756","600789"] 
        MATCH p2 = (a)-[:IN_CITY]-(c:CITY)-[:IN_PROVINCE]-(d:PROVINCE) 
        return p1,p2
        """

可视化后的结果如下：

![](../pictures/shandong_listed_companies.png)

### 2.2.图查询

关系图谱的一个很重要的应用就是为上层应用和服务提供图查询结构，并返回对应关联数据。
比如推荐系统、以及风控场景下的关联排查等


下面以本次的金融图谱举几个例子

### 2.2.1. 江苏省 最受 基金 欢迎（持仓最多）的上市公司


In [16]:
query1 = """
MATCH p = (a:FUND)-[:IN_PORTFOLIO]->(b:LISTED_COMPANY)-[:IN_CITY]-(c:CITY)-[:IN_PROVINCE]->(d:PROVINCE {province:"江苏"})  
RETURN c.city as city,b.company as company,count(DISTINCT a.fund_code) as num
ORDER BY num DESC
limit 10
"""

In [17]:
df = graph.run(query1).to_data_frame()
df.head(10)

""


#### 2.2.2. 上市公司高管董事之间的兼任情况

In [19]:
query2 = """
MATCH  (b:COMPANY)-[r1:其他|监事|高管|委员会成员|董事会成员|核心技术人员]->(d:MANAGER)
MATCH  (c:COMPANY)-[r2:其他|监事|高管|委员会成员|董事会成员|核心技术人员]->(d)
    WHERE  b.company<>c.company  
RETURN b.company as company_s,
       b.share_code as share_code_s ,
       r1.title as title_s,
       type(r1) as rel_s,
       c.company as company_d,
       c.share_code as share_code_d,
       d.manager_id as manager_id,
       r2.title as title_d,
       type(r2) as rel_d,
       d.name as name 
LIMIT 10 ;
"""

![](../pictures/same_manager.png)

In [20]:
df = graph.run(query2).to_data_frame()

In [21]:
df.head(2).T

""


#### 2.2.3.基金推荐

写个简单的基金推荐规则

根据某个基金持仓在不同行业的分布情况，基于此推荐与其持仓行业最相似的 Top5 基金。

In [22]:
fund_code = "001508"

In [23]:
query3 = """
MATCH p = (a:FUND)-[:IN_PORTFOLIO]->(b:COMPANY)-[:IN_INDUSTRY]-(c:INDUSTRY)  
    WHERE a.fund_code="{}"   
WITH  COLLECT(DISTINCT c.industry) as industry_base  
MATCH  (d:FUND)-[:IN_PORTFOLIO]->(e:COMPANY)-[:IN_INDUSTRY]-(f:INDUSTRY) 
WITH d.fund_code as fund_code, industry_base,collect(DISTINCT f.industry) as industry_target 
RETURN fund_code,
       toFloat(length([ind in industry_target where ind in industry_base ]))/length(industry_target) as ration
ORDER BY ration DESC
LIMIT 5
""".format(fund_code)

In [24]:
df = graph.run(query3).to_data_frame()

In [25]:
df.head()

""
